In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-21 21:26:50.978492: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 21:26:51.009489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [41]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [42]:
import adapters.composition as ac

domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel",with_head=False)
mlm_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/mlm-after-coral",with_head=False)
task_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/stacked-adapter-task-modefied",with_head=False)


In [43]:
model.add_adapter_fusion(ac.Fuse(mlm_adapter, task_adapter))
model.add_classification_head("mnli", num_labels=3)
adapter_setup_fuse = ac.Fuse( mlm_adapter, task_adapter)
model.train_adapter_fusion(adapter_setup_fuse)

model.active_adapters = ac.Stack(domain_adapter_coral, ac.Fuse(mlm_adapter, task_adapter))


In [44]:
fn.print_trainable_parameters(model)

trainable params: 11841597 || all params: 82644861 || trainable%: 14.32829199144034


In [45]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print(model.active_adapters)

distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.value.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.bias
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.wei

In [37]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.30465587044534415
F1 score before adaptation: 0.26615970008818535


In [46]:

reload(fn)
trainer = fn.train_model(model,"multi_block",source_data,target_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 0.6897, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.7791772484779358, 'eval_accuracy': 0.664382676147382, 'eval_f1': 0.66311905584895, 'eval_precision': 0.7322760128787381, 'eval_recall': 0.664382676147382, 'eval_runtime': 16.3222, 'eval_samples_per_second': 473.896, 'eval_steps_per_second': 14.826, 'epoch': 0.21}
{'loss': 0.6186, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.6697875261306763, 'eval_accuracy': 0.7182934712346477, 'eval_f1': 0.7191760574617739, 'eval_precision': 0.7326696435728212, 'eval_recall': 0.7182934712346477, 'eval_runtime': 16.5176, 'eval_samples_per_second': 468.289, 'eval_steps_per_second': 14.651, 'epoch': 3.62}
{'loss': 0.5553, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.666977047920227, 'eval_accuracy': 0.7235940530058177, 'eval_f1': 0.7240943111865201, 'eval_precision': 0.7394325887248321, 'eval_recall': 0.7235940530058177, 'eval_runtime': 16.4934, 'eval_samples_per_second': 468.976, 'eval_steps_per_second': 14.673, 'epoch': 3.84}


Overwriting existing adapter 'domain_adapter_telephone-travel'.
Overwriting existing adapter 'mlm-after-coral'.
Overwriting existing adapter 'stacked-adapter-task-modefied'.
Overwriting existing adapter fusion module 'mlm-after-coral,stacked-adapter-task-modefied'


{'train_runtime': 1666.2641, 'train_samples_per_second': 180.075, 'train_steps_per_second': 5.629, 'train_loss': 0.5890023318943438, 'epoch': 4.0}


In [40]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7130566801619433
F1 score before adaptation: 0.7149068389081805
